In [1]:
import json
import random

# Function to generate a random dietary restriction
def generate_dietary_restriction():
    options = ["none", "vegan", "vegetarian", "pescatarian", "halal"]
    return random.choice(options)

# Function to generate a nested document with an array of strings
def generate_nested_document():
    ingredients = [
        "celery", "wheat", "rye", "barley", "oats", "spelt", "kamut",
        "cabbage", "broccoli", "cauliflower", "kale", "brussel sprouts",
        "collard greens", "crustaceans", "eggs", "fish", "milk", "lupin",
        "molluscs", "mustard", "nuts", "peanuts", "sesame", "soy"
    ]

    # Randomly choose the size of the array
    array_size = random.randint(0, len(ingredients))
    # Randomly select ingredients for the array
    array_values = random.sample(ingredients, array_size)

    return {"allergens": array_values}

# Function to update each document in the JSON file
def update_json_file(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)

    for document in data:
        # Add the first property with a random dietary restriction
        document["diet_type"] = generate_dietary_restriction()
        # Add the second property with a nested document
        document["allergy"] = generate_nested_document()
        # Add admin boolean
        document['admin'] = False

    with open(file_path, 'w') as file:
        json.dump(data, file, indent=2)

# Replace 'your_file.json' with the actual file path
update_json_file('random_users2.json')

In [2]:
from pymongo import MongoClient

# Connect to MongoDB
client = MongoClient('mongodb://localhost:27017')
db = client['SafeBite']
collection = db['Users']

# Find duplicates based on 'user_name' field
pipeline = [
    {
        "$group": {
            "_id": {"user_name": "$user_name"},
            "unique_ids": {"$addToSet": "$_id"},
            "count": {"$sum": 1}
        }
    },
    {
        "$match": {
            "count": {"$gt": 1}
        }
    }
]

cursor = collection.aggregate(pipeline)

# Remove duplicates
for doc in cursor:
    # Keep the first document and delete the rest
    ids_to_keep = doc['unique_ids'][0]
    ids_to_remove = doc['unique_ids'][1:]

    collection.delete_many({"_id": {"$in": ids_to_remove}})

print("Duplicates removed successfully.")

# Close the MongoDB connection
client.close()

Duplicates removed successfully.
